# 3-3. 간단한 보상 점수 실험 (naver-news-dpo)

이 노트북에서는 Hugging Face의 `dev7halo/naver-news-dpo` 데이터셋을 활용해서

- 실제 DPO(선호 학습) 데이터 구조를 확인하고
- 직접 **A/B 중 더 좋은 답변을 선택**해 보고
- 그 결과를 **원래 데이터의 preference(chosen)** 과 비교합니다.

이 과정을 통해,

> *"사람 선호 → 보상 점수(Reward) → 모델 학습"*

이라는 Alignment의 핵심 아이디어를 직관적으로 이해하는 것이 목표입니다.

---
### 오늘 노트북에서 할 것
1. 데이터셋 로딩 및 구조 확인
2. 랜덤으로 예시 하나 뽑아서 `prompt / Answer A / Answer B` 읽어보기
3. 내가 A/B 중 어느 쪽을 더 선호하는지 선택하기
4. 데이터셋이 가진 정답(chosen)과 **내 선택의 일치율(agreement)** 계산하기
5. 규칙 기반 `tiny_reward()` 함수를 정의해서, 각 답변에 점수를 매겨보기


In [2]:
# 라이브러리 설치
!pip install -q datasets pandas

In [1]:
from datasets import load_dataset
import random
import pandas as pd

# dev7halo/naver-news-dpo 데이터셋 로딩, 다른 dpo 데이터셋도 유사한 방법으로 로딩 가능
ds = load_dataset("dev7halo/naver-news-dpo", split="train")

print(ds)
print("컬럼:", ds.column_names)
print("샘플 개수:", len(ds))


Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 1638
})
컬럼: ['prompt', 'chosen', 'rejected']
샘플 개수: 1638


### 예시 한 개를 보기 좋게 출력하는 함수

- `swap=False`이면 A=chosen, B=rejected
- `swap=True`이면 A=rejected, B=chosen 으로 섞어서 보여줍니다.
  - 이렇게 하면 항상 chosen이 A로 보이는 편향을 줄일 수 있습니다.


In [2]:
def show_example(example, swap=False):
    """하나의 예시를 프롬프트와 함께 A/B 두 답변으로 출력합니다.

    example: ds에서 뽑은 한 row (dict-like)
    swap=False이면 A=chosen, B=rejected
    swap=True이면 A=rejected, B=chosen

    return: 데이터셋 기준으로 더 선호된 답이 어느 쪽(A/B)인지 라벨
    """
    prompt = example["prompt"]
    chosen = example["chosen"]
    rejected = example["rejected"]

    if not swap:
        answer_A = chosen
        answer_B = rejected
        correct_label = "A"   # 데이터셋이 선호한 쪽
    else:
        answer_A = rejected
        answer_B = chosen
        correct_label = "B"

    print("=" * 80)
    print("[Prompt]")
    print(prompt.strip(), "\n")

    print("[Answer A]")
    print(answer_A.strip(), "\n")

    print("[Answer B]")
    print(answer_B.strip(), "\n")

    return correct_label


## 인터랙티브 A/B 선호 선택

아래 셀을 실행하면 여러 번의 **ROUND**가 진행됩니다.

각 라운드에서:
1. 프롬프트와 두 개의 답변 A/B가 출력됩니다.
2. 여러분은 `A`, `B` 중 어느 쪽이 더 나은지 입력합니다.
3. 데이터셋의 원래 선호(chosen)와 일치하는지 여부를 바로 확인할 수 있습니다.


In [5]:
try:
    num_rounds = int(input("몇 번 정도 비교해볼까요? (예: 5): ").strip() or "5")
except ValueError:
    num_rounds = 5
    print("정수가 아니어서 기본값 5로 설정합니다.")

results = []  # 각 라운드 결과 기록용

for round_idx in range(1, num_rounds + 1):
    print(f"\n\n############ ROUND {round_idx} ############\n")

    # 무작위 예시 하나 선택
    ex = ds[random.randint(0, len(ds)-1)]

    # A/B 순서를 랜덤하게 섞어서 보여준다 (편향 방지)
    swap = random.random() < 0.5
    correct_label = show_example(ex, swap=swap)

    # 사용자 입력 받기
    while True:
        user_choice = input("당신이 더 선호하는 답은? (A/B, 건너뛰기: S): ").strip().upper()
        if user_choice in ["A", "B", "S"]:
            break
        print("A, B, S 중 하나를 입력해주세요.")

    if user_choice == "S":
        print("이 라운드는 건너뜁니다.")
        continue

    # 기록
    agree = (user_choice == correct_label)

    # 현재 A/B에 실제로 어떤 텍스트가 들어갔는지도 기록해 둡니다.
    if not swap:
        answer_A = ex["chosen"]
        answer_B = ex["rejected"]
    else:
        answer_A = ex["rejected"]
        answer_B = ex["chosen"]

    results.append(
        {
            "round": round_idx,
            "prompt": ex["prompt"],
            "answer_A": answer_A,
            "answer_B": answer_B,
            "dataset_preferred": correct_label,  # DPO 기준 더 선호된 쪽(A/B)
            "my_choice": user_choice,
            "agree_with_dataset": agree,
        }
    )

    print(f"\n[결과] 데이터셋이 선호한 쪽: {correct_label}, 나의 선택: {user_choice}")
    if agree:
        print("→ ✅ 나의 선택이 DPO preference와 일치합니다.")
    else:
        print("→ ❌ 나의 선택이 DPO preference와 다릅니다.")




############ ROUND 1 ############

[Prompt]
주어진 기사 제목을 이용해 신문기사를 작성하세요. 기사 제목: 재창업 자금지원에도…5년후 좀비 수두룩 

[Answer A]
중진공 2016 2021년 지원 조사 3년후부터 70%가 고용성과 급감 매출도 초기 대비 10%로 떨어져 기업 단계별 지원 대책 마련 필요 서울경제 정부 자금을 제공 받고 재창업에 나섰던 기업 10곳 중 7곳 가량이 지원을 받은 지 3년 이후에는 고용 인원이 1명 이하로 떨어지는 것으로 나타났다. 이 기간 동안 휴·폐업 업체들이 속출하며 기업 전체 매출액도 지원 당시 대비 10% 수준으로 고꾸라졌다. 정부 지원을 발판 삼아 재도전에 나섰지만 ‘창업 데스밸리’를 넘지 못하고 현재 이름만 유지하는 기업들이 수두룩하다는 지적이 나온다. 정부 자금이 ‘밑 빠진 독에 물 붓기’가 되지 않으려면 후속 지원이 필요하다고 재도전 기업인들은 입을 모은다. 6일 더불어민주당 소속 이성만 의원실과 중소벤처기업진흥공단에 따르면 지난 2016년부터 총 4021곳이 재창업 자금 지원을 받았다. 재도전 지원에는 연간 1000억 1200억 원이 자금이 집행돼 지난 6년간 총 6600억원이 투입됐다. ‘실패 기업인’에게 1억 원 가량의 자금을 빌려주는 형식으로 거치 기간 3년을 두고 6년 이내에 대출을 상환하도록 한다. 연간 100억 원대가 집행되다 박근혜 정부 시절인 2016년부터 1000억대로 확대됐다. 지원 규모는 늘었지만 성과는 기대에 미치지 못하고 있다는 평가가 적지 않다. 실제 지난 2016년 지원 받은 기업 526곳 가운데 올 5월 기준 종업원 수가 하나도 없는 업체는 63.6% 335곳 에 달한다. 지원 당시 10 20여 명의 직원이 있었지만 현재 아무도 남아 있지 않는 회사가 상당수다. 아울러 현 종업원이 1명에 그치는 곳 또한 6.08% 32곳 로 집계된다. 고용 창출 능력이 희박한 곳이 69.8%에 달하는 셈으로 이 경우 사실상 폐업에 가깝다는 게 업계의 지배적인 해

## 선택 결과 요약 보기

여기서는 지금까지의 라운드에서:
- 내가 선택한 답(my_choice)
- 데이터셋 기준 선호(dataset_preferred)
- 둘이 일치하는지 여부(agree_with_dataset)
를 표로 정리하고, **agreement rate** 를 계산합니다.


In [6]:
if len(results) == 0:
    print("기록된 라운드가 없습니다. 위 셀을 다시 실행해보세요.")
else:
    df = pd.DataFrame(results)
    display(df[["round", "dataset_preferred", "my_choice", "agree_with_dataset"]])

    agreement_rate = df["agree_with_dataset"].mean()
    print(f"\n📊 나의 선택과 DPO 데이터셋의 선호(chosen)가 일치한 비율: {agreement_rate:.2%}")

    print("\n내가 더 자주 고른 답 (A/B):")
    print(df["my_choice"].value_counts())


,round,dataset_preferred,my_choice,agree_with_dataset
0,1,A,A,True
1,2,A,A,True
2,3,B,A,False



📊 나의 선택과 DPO 데이터셋의 선호(chosen)가 일치한 비율: 66.67%

내가 더 자주 고른 답 (A/B):
my_choice
A    3
Name: count, dtype: int64


## 규칙 기반 tiny_reward 함수로 점수 매겨보기

이제 아주 단순한 규칙 기반 보상 함수를 하나 만들어서,
`answer_A`, `answer_B` 에 점수를 매겨보고,
우리의 규칙 기반 보상 함수가 사람/데이터셋의 선호와 얼마나 일치하는지도 확인해봅니다.


In [7]:
# 아주 단순한 규칙 기반 '보상함수' 예시
positive_keywords = ["추천", "도움", "도움이 될", "좋습니다", "좋아요", "권장"]
detail_keywords = ["요약", "핵심", "정리", "설명", "자세히"]

negative_keywords = ["그냥", "막", "대충", "상관없다"]
risky_keywords = ["폭발물", "마약", "죽여", "테러"]  # 이 데이터셋에는 거의 없겠지만 예시로 둠

def tiny_reward(text: str) -> float:
    """규칙 기반 보상 함수 예시.
    실제 RLHF의 보상모델은 이런 기준을 신경망이 직접 학습하는 버전이라고 볼 수 있습니다.
    """
    score = 0.0

    # 긍정/상세 표현에 가산점
    if any(kw in text for kw in positive_keywords):
        score += 1.0
    if any(kw in text for kw in detail_keywords):
        score += 0.5

    # 부정/무책임 표현에 감점
    if any(kw in text for kw in negative_keywords):
        score -= 0.5
    if any(kw in text for kw in risky_keywords):
        score -= 2.0

    # 너무 짧으면 약간 감점
    if len(text) < 40:
        score -= 0.5

    return score

print("tiny_reward 함수 정의 완료")


tiny_reward 함수 정의 완료


In [ ]:
if len(results) == 0:
    print("먼저 위의 인터랙티브 실습을 진행해주세요.")
else:
    print("최근 라운드 몇 개에 대해 tiny_reward 점수 비교:")

    for _, row in df.tail(5).iterrows():
        ra = tiny_reward(row["answer_A"])
        rb = tiny_reward(row["answer_B"])
        print("\n=== Round", row["round"], "===")
        print("dataset_preferred:", row["dataset_preferred"])
        print("my_choice:", row["my_choice"])
        print("tiny_reward(A):", ra)
        print("tiny_reward(B):", rb)
        better = "A" if ra > rb else "B" if rb > ra else "tie"
        print("tiny_reward 기준으로 더 좋은 쪽:", better)
        
        if better == row["dataset_preferred"]:
            print("→ 규칙 기반 보상함수와 DPO 선호가 일치합니다.")
        else:
            print("→ 규칙 기반 보상함수와 DPO 선호가 다릅니다.")


최근 라운드 몇 개에 대해 tiny_reward 점수 비교:

=== Round 1 ===
dataset_preferred: A
my_choice: A
tiny_reward(A): 0.0
tiny_reward(B): 0.0
tiny_reward 기준으로 더 좋은 쪽: tie
→ 규칙 기반 보상함수와 DPO 선호가 다릅니다.

=== Round 2 ===
dataset_preferred: A
my_choice: A
tiny_reward(A): 0.0
tiny_reward(B): -0.5
tiny_reward 기준으로 더 좋은 쪽: A
→ 규칙 기반 보상함수와 DPO 선호가 일치합니다.

=== Round 3 ===
dataset_preferred: B
my_choice: A
tiny_reward(A): 0.0
tiny_reward(B): 0.5
tiny_reward 기준으로 더 좋은 쪽: B
→ 규칙 기반 보상함수와 DPO 선호가 일치합니다.


: 

### 마무리

- `dev7halo/naver-news-dpo` 는 실제 DPO/선호 학습에 사용 가능한 형태의 데이터셋입니다.
- 우리는 여기서 **prompt + (chosen, rejected)** 구조를 보고,
  - A/B 중 어느 쪽을 더 선호하는지 직접 선택해 보았습니다.
  - 나의 선택과 데이터셋의 `chosen` 이 얼마나 일치하는지 확인했습니다.
- 또한 아주 단순한 규칙 기반 `tiny_reward()` 를 정의해서,
  - 텍스트에 점수를 매기고,
  - 이 점수가 DPO 선호와 얼마나 일치하는지 살펴보았습니다.

실제 RLHF / Reward Model 학습에서는,

> 이런 선호 데이터(Preference)를 대규모로 모으고,
> 신경망을 이용해 **보상 함수를 근사**한 다음,
> 그 보상 함수를 기준으로 LLM의 파라미터를 업데이트합니다.

